## 1. drawfit 전체 상품의 좋아요 수 크롤링

In [9]:
# drawfit 상품 정보 불러오기
import pandas as pd

# 1. 파일 경로 지정
file_path_drawfit = "./Data/drawfit_label.csv"

# 2. CSV 파일을 DataFrame으로 불러오기
df_drawfit_label = pd.read_csv(file_path_drawfit, encoding="utf-8")  # 보통 utf-8, 혹은 cp949 필요할 수 있음

# 3. 데이터 확인
print(df_drawfit_label.head())

   goodsNo                           goodsName  \
0  5275962  [패션플래닛X드로우핏] 세미 와이드 치노 팬츠 [6color]   
1  1760177                 도브 울 블루종 자켓 [BLACK]   
2  5384864       핀 스트라이프 크롭 블루종 자켓 [DARK NAVY]   
3  5290700   [서강준PICK] 레이어 미니멀 스웨이드 자켓 [BROWN]   
4  5384863        핀 스트라이프 크롭 블루종 자켓 [CHARCOAL]   

                               goodsLinkUrl  \
0  https://www.musinsa.com/products/5275962   
1  https://www.musinsa.com/products/1760177   
2  https://www.musinsa.com/products/5384864   
3  https://www.musinsa.com/products/5290700   
4  https://www.musinsa.com/products/5384863   

                                           thumbnail displayGenderText  \
0  https://image.msscdn.net/images/goods_img/2025...                남성   
1  https://image.msscdn.net/images/goods_img/2021...                남성   
2  https://image.msscdn.net/images/goods_img/2025...                남성   
3  https://image.msscdn.net/images/goods_img/2025...                남성   
4  https://image.msscdn.net/images/goods_img/2

In [10]:
# df_label의 goodsNo 컬럼을 리스트로 변환
drawfit_goods_list = df_drawfit_label['goodsNo'].tolist()

In [11]:
# 리스트 요소 개수 확인
len(drawfit_goods_list)

723

In [ ]:
import requests
import pandas as pd

# 1. API URL
url = "https://like.musinsa.com/like/api/v2/liketypes/goods/counts"

# 2. 요청 헤더
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Origin": "https://www.musinsa.com",
    "Referer": "https://www.musinsa.com/",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    # "Cookie": "여기에 최신 쿠키값 입력"
}

# 3. relationIds 
batch_size = 100
results_drawfit = []

# 100개 단위로 나눠서 API 호출
for i in range(0, len(drawfit_goods_list), batch_size):
    batch_drawfit = drawfit_goods_list[i:i + batch_size]
    payload = {
        "relationIds": batch_drawfit
    }
    
    response_drawfit = requests.post(url, headers=headers, json=payload)
    
    if response_drawfit.status_code == 200:
        data_drawfit = response_drawfit.json()
        try:
            items = data_drawfit["data"]["contents"]["items"]
            results_drawfit.extend(items)
        except KeyError:
            print("파싱 실패:", data_drawfit)
    else:
        print("요청 실패:", response_drawfit.status_code, response_drawfit.text)

# 4. 최종 DataFrame으로 변환
df_drawfit = pd.DataFrame(results_drawfit)

In [13]:
print('df_drawfit 개수: ', len(df_drawfit))

df_drawfit 개수:  723


In [14]:
df_drawfit

,likeType,relationId,count,liked
0,GOODS,5275962,4793,False
1,GOODS,1760177,18160,False
2,GOODS,5384864,2160,False
3,GOODS,5290700,1625,False
4,GOODS,5384863,1024,False
...,...,...,...,...
718,GOODS,1333401,781,False
719,GOODS,1305433,2124,False
720,GOODS,1130425,564,False
721,GOODS,607925,3220,False


## 2. standarderror 전체 상품의 좋아요 수 크롤링

In [ ]:
import pandas as pd

# 1. 파일 경로 지정
file_path_standard = "./Data/standard_label.csv"

# 2. CSV 파일을 DataFrame으로 불러오기
df_standard_label = pd.read_csv(file_path_standard, encoding="utf-8")  # 보통 utf-8, 혹은 cp949 필요할 수 있음

# 3. 데이터 확인
print(df_standard_label.head())

In [16]:
# df_label의 goodsNo 컬럼을 리스트로 변환
standard_goods_list = df_standard_label['goodsNo'].tolist()

In [17]:
# 리스트 요소 개수 확인
len(standard_goods_list)

291

In [18]:
# 3. relationIds 
batch_size = 100
results_standard = []

# 100개 단위로 나눠서 API 호출
for i in range(0, len(standard_goods_list), batch_size):
    batch_standard = standard_goods_list[i:i + batch_size]
    payload = {
        "relationIds": batch_standard
    }
    
    response_standard = requests.post(url, headers=headers, json=payload)
    
    if response_standard.status_code == 200:
        data_standard = response_standard.json()
        try:
            items = data_standard["data"]["contents"]["items"]
            results_standard.extend(items)
        except KeyError:
            print("파싱 실패:", data_standard)
    else:
        print("요청 실패:", response_standard.status_code, response_standard.text)

# 4. 최종 DataFrame으로 변환
df_standard = pd.DataFrame(results_standard)

In [19]:
print('df_standard 개수: ', len(df_standard))

df_standard 개수:  291


## 3. seez 전체 상품의 좋아요 수 크롤링

In [ ]:
import pandas as pd

# 1. 파일 경로 지정
file_path_seez = "./Data/seez_label.csv"

# 2. CSV 파일을 DataFrame으로 불러오기
df_seez_label = pd.read_csv(file_path_seez, encoding="utf-8")  # 보통 utf-8, 혹은 cp949 필요할 수 있음

# 3. 데이터 확인
print(df_seez_label.head())

In [22]:
# df_label의 goodsNo 컬럼을 리스트로 변환
seez_goods_list = df_seez_label['goodsNo'].tolist()

In [ ]:
# 리스트 요소 개수 확인
len(seez_goods_list)

159

In [25]:
# 3. relationIds 
batch_size = 100
results_seez = []

# 100개 단위로 나눠서 API 호출
for i in range(0, len(seez_goods_list), batch_size):
    batch_seez = seez_goods_list[i:i + batch_size]
    payload = {
        "relationIds": batch_seez
    }
    
    response_seez = requests.post(url, headers=headers, json=payload)
    
    if response_seez.status_code == 200:
        data_seez = response_seez.json()
        try:
            items = data_seez["data"]["contents"]["items"]
            results_seez.extend(items)
        except KeyError:
            print("파싱 실패:", data_seez)
    else:
        print("요청 실패:", response_seez.status_code, response_seez.text)

# 4. 최종 DataFrame으로 변환
df_seez = pd.DataFrame(results_seez)

In [26]:
print('df_seez 개수: ', len(df_seez))

df_seez 개수:  159


## 4. kiimuir 전체 상품의 좋아요 수 크롤링

In [ ]:
import pandas as pd

# 1. 파일 경로 지정
file_path_kiimuir = "./Data/kiim_label.csv"

# 2. CSV 파일을 DataFrame으로 불러오기
df_kiimuir_label = pd.read_csv(file_path_kiimuir, encoding="utf-8")  # 보통 utf-8, 혹은 cp949 필요할 수 있음

# 3. 데이터 확인
print(df_kiimuir_label.head())

In [30]:
# df_label의 goodsNo 컬럼을 리스트로 변환
kiimuir_goods_list = df_kiimuir_label['goodsNo'].tolist()

In [31]:
# 리스트 요소 개수 확인
len(kiimuir_goods_list)

379

In [32]:
# 3. relationIds 
batch_size = 100
results_kiimuir = []

# 100개 단위로 나눠서 API 호출
for i in range(0, len(kiimuir_goods_list), batch_size):
    batch_kiimuir = kiimuir_goods_list[i:i + batch_size]
    payload = {
        "relationIds": batch_kiimuir
    }
    
    response_kiimuir = requests.post(url, headers=headers, json=payload)
    
    if response_kiimuir.status_code == 200:
        data_kiimuir = response_kiimuir.json()
        try:
            items = data_kiimuir["data"]["contents"]["items"]
            results_kiimuir.extend(items)
        except KeyError:
            print("파싱 실패:", data_kiimuir)
    else:
        print("요청 실패:", response_kiimuir.status_code, response_kiimuir.text)

# 4. 최종 DataFrame으로 변환
df_kiimuir = pd.DataFrame(results_kiimuir)

In [33]:
print('df_kiimuir 개수: ', len(df_kiimuir))

df_kiimuir 개수:  379


In [ ]:
# # csv 저장
# df_standard.to_csv('Data/standard_counts.csv', index=False)
# df_drawfit.to_csv('Data/drawfit_counts.csv', index=False)
# df_kiimuir.to_csv('Data/kiim_counts.csv', index=False)
# df_seez.to_csv('Data/seez_counts.csv', index=False)